## 1. Convert probe matrix to gene matrix 

We prefer taking the maximum count among probes as the count of that gene. This can be done with `gene_matrix_from_probe_matrix.m` function by passing `max_probe_in_cell` to `map_method` argument. (There is yet-to-be-tested python version of this in "Python" branch of the gitlab repo) Let's say this matrix is named `gene_count`. 

Then sort the matrix in a descending order. Inspect its barcode rank plot, `loglog(gene_count.sum(axis=1))`, and choose $U$ and $T$ cutoffs to identify real/putative cells and ambient barcodes. Please refer to the "Cell Calling" supplemental text and the associated figure. At this point, you should be able to plug-in to the following pipeline.

In [ ]:
import numpy as np
import scipy

import statsmodels.api as sm
from statsmodels.stats.multitest import multipletests

In [ ]:
# filter genes without count (in ambient barcodes)
ambient_count = gene_count[T:] 
total_gene_count_ambient = ambient_count.sum(axis=0)
nonzero_ind = np.flatnonzero(total_gene_count_ambient)

# break gene_count matrix into the three count matrices
real_count = gene_count[:U, nonzero_ind]
putative_count = gene_count[U:T, nonzero_ind]
ambient_count = gene_count[T:, nonzero_ind] 

# get normalized count matrix 
ambient_freq = ambient_count[:, nonzero_ind] / ambient_count[:, nonzero_ind].sum(axis=1, keepdims=True)                                                                                   

## 2. Find proportionality constant $\alpha$ between mean and variance of normalized count from ambient barcodes

In [ ]:
ambient_mean = ambient_freq.mean(axis=0)
ambient_var = ambient_freq.var(axis=0)

res = sm.OLS(ambient_var, ambient_mean).fit()
slope = res.params.item()

lower_cutoff = int(np.ceil(1/slope))
'''Both real and putative counts should have per-barcode total count greater than lower_cutoff'''
# real_ind_to_keep = np.flatnonzero(real_count.sum(axis=1) > lower_cutoff)
# real_count = real_count[real_ind_to_keep]
# putative_ind_to_keep = np.flatnonzero(putative_count.sum(axis=1) > lower_cutoff)
# putative_count = putative_count[putative_ind_to_keep]
ambient_ind_to_keep = np.flatnonzero(ambient_count.sum(axis=1) > lower_cutoff)
ambient_count = ambient_count[ambient_ind_to_keep]

## 3. Calculate $p$-values using negative binomial distribution
We are using $\alpha$ to obtain variance in *count* from variance in *frequency*.

In [ ]:
def collect_pvalues(count_matrix, prior_p, slope, num_sim=1000):
    pvals_total = []
    for barcode in count_matrix:
        total_count = barcode.sum()

        mean = prior_p * total_count
        var = (prior_p * slope) * (total_count ** 2) 

        phi = mean**2 / (var - mean)
        p = mean / var

        Lb = scipy.stats.nbinom.logpmf(barcode, phi, p).sum()

        samples = scipy.stats.nbinom.rvs(phi, p, size=(num_sim, len(prior_p)))
        Lsamples = scipy.stats.nbinom.logpmf(samples, phi, p).sum(axis=1)
        pval = (np.count_nonzero(Lsamples < Lb) + 1) / (num_sim + 1)
        pvals_total.append(pval)   
        
    return np.array(pvals_total)

In [ ]:
putative_pvals = collect_pvalues(putative_count, ambient_mean, slope)

'''trt setting this error rate, alpha, as low as possible (e.g., 0.002)'''
putative_reject, putative_pvals_corrected = multipletests(putative_pvals, alpha=0.05, method='fdr_bh')[:2]  

gene_count_called = np.vstack((real_count, putative_count[putative_reject]))